In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import queue
import contextlib
import time
import PyQt5

In [2]:
# pip install PyQt5

## Import the logs (backlog + incoming) from .CSV and create DataFrame

In [3]:
backlog_df = pd.read_csv('defectRemediation_log_simulation_vPrototype.csv', index_col='ID')
backlog_df['Timestamp'] = pd.to_datetime(backlog_df['Timestamp'], format='%Y-%d-%m %H:%M:%S')
# backlog_df
incoming_log_df = pd.read_csv('defectRemediation_log_incoming_simulation_vPrototype.csv', index_col='ID')
incoming_log_df['Timestamp'] = pd.to_datetime(incoming_log_df['Timestamp'], format='%Y-%d-%m %H:%M:%S')
incoming_log_df

,Defect_ID,Control_Type,State,Date,Timestamp
ID,,,,,
117,30,AUTH18,new,2025-30-05,2025-05-30 23:14:52
118,31,AUTH42,new,2025-30-05,2025-05-30 23:15:16
119,30,AUTH18,assign,2025-31-05,2025-05-31 09:14:36
120,32,ACC28,new,2025-31-05,2025-05-31 09:15:37
121,31,AUTH42,assign,2025-31-05,2025-05-31 09:20:56
122,33,AUTH18,new,2025-31-05,2025-05-31 09:22:32
123,34,AUTH18,new,2025-31-05,2025-05-31 09:22:45
124,35,AUTH18,new,2025-31-05,2025-05-31 09:23:09
125,30,AUTH18,in-progress,2025-31-05,2025-05-31 12:13:54


In [4]:
hours_backlog = []
hours_incoming = []
for row in backlog_df.iterrows():
    hours_backlog.append(backlog_df.loc[row[0]]['Timestamp'].hour)
for row in incoming_log_df.iterrows():
    hours_incoming.append(incoming_log_df.loc[row[0]]['Timestamp'].hour)
backlog_df['Hour'] = hours_backlog
incoming_log_df['Hour'] = hours_incoming
incoming_log_df

,Defect_ID,Control_Type,State,Date,Timestamp,Hour
ID,,,,,,
117,30,AUTH18,new,2025-30-05,2025-05-30 23:14:52,23
118,31,AUTH42,new,2025-30-05,2025-05-30 23:15:16,23
119,30,AUTH18,assign,2025-31-05,2025-05-31 09:14:36,9
120,32,ACC28,new,2025-31-05,2025-05-31 09:15:37,9
121,31,AUTH42,assign,2025-31-05,2025-05-31 09:20:56,9
122,33,AUTH18,new,2025-31-05,2025-05-31 09:22:32,9
123,34,AUTH18,new,2025-31-05,2025-05-31 09:22:45,9
124,35,AUTH18,new,2025-31-05,2025-05-31 09:23:09,9
125,30,AUTH18,in-progress,2025-31-05,2025-05-31 12:13:54,12


### Append backlogged defects to processing queue

In [5]:
processing_queue = queue.Queue()

sub_backlog_df = backlog_df.query('State == "closed"')
for row in sub_backlog_df.iterrows():
    with contextlib.suppress(IndexError):
        processing_queue.put(sub_backlog_df.loc[row[0]]['Defect_ID'], block=False)

sub_backlog_df
# processing_queue.get(block=False)

,Defect_ID,Control_Type,State,Date,Timestamp,Hour
ID,,,,,,
6,1,ACC03,closed,2025-16-05,2025-05-16 00:34:56,0
11,2,ACC17,closed,2025-16-05,2025-05-16 01:59:42,1
18,6,AUTH42,closed,2025-16-05,2025-05-16 02:52:32,2
23,4,AUTH18,closed,2025-16-05,2025-05-16 03:54:27,3
24,5,ACC03,closed,2025-16-05,2025-05-16 04:17:45,4
25,3,ACC28,closed,2025-16-05,2025-05-16 04:23:20,4
28,7,ACC17,closed,2025-16-05,2025-05-16 04:55:35,4
35,8,ACC03,closed,2025-17-05,2025-05-17 11:13:54,11
39,9,ACC03,closed,2025-17-05,2025-05-17 15:43:35,15


In [6]:
# processing_queue.get(block=False)

## Creating Delta table to track time b/w state changes

In [7]:
deltas_df = pd.DataFrame(columns=['Defect_ID','Control_Type','Delta_New_Assign','Delta_Assign_InProgress','Delta_InProgress_Closed', 'Delta_New_Closed'])

In [8]:
try: # deltas_df contains some entries, add onto it
    Last_ID_processed = deltas_df.index[-1]
    index = Last_ID_processed + 1
except IndexError: # deltas_df is empty, start from beginning
    index = 0 

In [9]:
# master dictionary with tracked empirical data to build distributions
empirical_dict = {}
incoming_dict = {}
figure_id_dict = {}
figure_id = 0
control_types = backlog_df['Control_Type'].unique()
for control_type in control_types:
    empirical_dict[control_type] = {'incoming_per_hour': [],
                                    'delta_new_assign': [],
                                    'delta_assign_inprogress': [],
                                    'delta_inprogress_closed': [],
                                    'delta_new_closed': []
                                   }
    incoming_dict[control_type] = {}
    figure_id_dict[control_type] = figure_id
    figure_id += 1

In [10]:
%matplotlib qt

In [11]:
t_end = 50
# fig, axs = plt.subplots (1, len(empirical_dict.keys()), figsize=(16,20))
fig, ax = plt.subplots()
for i in range(t_end):
    #### PULL DEFECT TO PROCESS - IN CHRONOLOGICAL ORDER OF STATE == "CLOSED" ####
    if processing_queue.qsize() > 0:
        with contextlib.suppress(queue.Empty):
            defect_id = processing_queue.get(block=False)
            
        #### Recover timestamps for each state change ####
        timestamp_new = backlog_df.loc[(backlog_df['Defect_ID']==defect_id) & (backlog_df['State']=='new')]['Timestamp'].iloc[0] # OR backlog_df.query('(Defect_ID == defect_id) and (State == "new")')['Timestamp'].iloc[0]
        timestamp_assign = backlog_df.loc[(backlog_df['Defect_ID']==defect_id) & (backlog_df['State']=='assign')]['Timestamp'].iloc[0]
        timestamp_inprogress = backlog_df.loc[(backlog_df['Defect_ID']==defect_id) & (backlog_df['State']=='in-progress')]['Timestamp'].iloc[0]
        timestamp_closed = backlog_df.loc[(backlog_df['Defect_ID']==defect_id) & (backlog_df['State']=='closed')]['Timestamp'].iloc[0]
    
        # time between assign and new (hrs)
        delta_new_assign = timestamp_assign - timestamp_new
        delta_new_assign = round(delta_new_assign.total_seconds()/3600, 3)
        
        # time between in-progress and assign (hrs)
        delta_assign_inprogress = timestamp_inprogress - timestamp_assign
        delta_assign_inprogress = round(delta_assign_inprogress.total_seconds()/3600, 3)
        
        # time between closed and in-progress (hrs)
        delta_inprogress_closed = timestamp_closed - timestamp_inprogress
        delta_inprogress_closed = round(delta_inprogress_closed.total_seconds()/3600, 3)

        # time between closed and new (hrs)
        delta_new_closed = sum([delta_new_assign, delta_assign_inprogress, delta_inprogress_closed])
    
        # append row to df and iterate index
        control_type = backlog_df.loc[backlog_df['Defect_ID'] == defect_id].iloc[0]['Control_Type']
        deltas_df.loc[index] = [defect_id, control_type, delta_new_assign, delta_assign_inprogress, delta_inprogress_closed, delta_new_closed]
        # delta_new_closed = deltas_df.loc[index, ['Delta_New_Assign','Delta_Assign_InProgress', 'Delta_InProgress_Closed']].sum()
        # deltas_df.loc[index, 'Delta_New_Closed'] = delta_new_closed
        
        
        #### UPDATE PLOTS ####
        # time.sleep(2)
        # print(i)
        # axs[figure_id_dict[control_type]].plot(delta_new_closed)
        if control_type == "ACC03":
            try:
                last_value = ax.lines[-1].get_xdata()[-1]
            except IndexError:
                last_value = 0
            # print(last_value, delta_new_closed)
            ax.plot(last_value+1, delta_new_closed, 'ko', markersize=6)

        index += 1

plt.show()

In [12]:
ax.lines[-1]

In [13]:
figure_id_dict

{'ACC03': 0, 'ACC17': 1, 'ACC28': 2, 'AUTH18': 3, 'AUTH42': 4}

In [14]:
# fig, ax = plt.subplots()
# ax.plot([1, 2, 3],[42, 28, 36])
# try:
#     last_value = plt.gca().lines[0].get_xdata()[-1]
# except IndexError:
#     last_value = 0

# last_value
    
